In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import matplotlib.pyplot as plt 
import numpy as np

from scipy.cluster.hierarchy import single, complete, average, ward, dendrogram
from sklearn.cluster import AgglomerativeClustering
## Library 1

from sktime.distances.elastic_cython import dtw_distance
from scipy.cluster.hierarchy import fcluster

from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis


ModuleNotFoundError: No module named 'requests'

In [ ]:
companies = ['TSLA', 'MSFT', 'FB', 'ADBE' , 'BA', 'T', 'COKE', 'GOOG', 'HON']

yahoo_financials = YahooFinancials(companies)

data = yahoo_financials.get_historical_price_data(start_date='2019-01-01',
                                                  end_date='2019-12-31',
                                                  time_interval='daily')

prices_df = pd.DataFrame({
    a: {x['formatted_date']: x['adjclose'] for x in data[a]['prices']} for a in companies
})

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
prices_df.plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

cols = list(prices_df.columns)
for col in cols:
    prices_df[col] = (prices_df[col]) - np.mean((prices_df[col]))
    
prices_df.plot(ax=ax)

In [ ]:
chunk_size = int(prices_df.shape[0] / 30)
i = 0

og_prices_dict = {}

for start in range(0, prices_df.shape[0], chunk_size):
    df_subset = prices_df.iloc[start:start + chunk_size]
    df_subset.columns = ['TSLA'+str(i), 'MSFT'+str(i), 'FB'+str(i), 'ADBE'+str(i), 'BA'+str(i), 'T'+str(i), 'COKE'+str(i), 'GOOG'+str(i), 'HON'+ str(i)]
    new_prices_dict = df_subset.to_dict('series')
    og_prices_dict.update(new_prices_dict)
    i = i + 1
print(og_prices_dict) 

In [ ]:
distanceMatrix = np.empty((len(og_prices_dict), len(og_prices_dict),))
distanceMatrix[:] = np.nan

distanceMatrixDictionary = {}

for index1, (filter1, trajectory1) in enumerate(og_prices_dict.items()):
    
     for index2, (filter2, trajectory2) in enumerate(og_prices_dict.items()):
            
                unionFilter = filter1 + filter2

                trajectory1 = np.asarray(trajectory1)
                trajectory2 = np.asarray(trajectory2)
                trajectory1 = trajectory1.reshape((-1, 1))
                trajectory2 = trajectory2.reshape((-1, 1))
                metric = dtw_distance(trajectory1, trajectory2)
                distanceMatrix[index1][index2] = metric
                distanceMatrixDictionary[unionFilter] = metric

In [ ]:
def hierarchical_clustering(distanceMatrix, method='ward'):
    if method == 'complete':
        Z = complete(distanceMatrix)
    if method == 'single':
        Z = single(distanceMatrix)
    if method == 'average':
        Z = average(distanceMatrix)
    if method == 'ward':
        Z = ward(distanceMatrix)
    
    fig = plt.figure(figsize=(16, 8))
    dn = dendrogram(Z)
    plt.title(f"Dendrogram for {method}-linkage with correlation distance")
    plt.show()
    
    return Z

linkage_matrix = hierarchical_clustering(distanceMatrix)

In [ ]:
cluster_labels = fcluster(linkage_matrix, 8, criterion='maxclust')
f = np.expand_dims(distanceMatrix, 1)
cluster_labels

In [ ]:
newdf = pd.DataFrame.from_dict(og_prices_dict)
newdf.reset_index(inplace=True)
mapping = {}
 
for index1, (filter1, trajectory1) in enumerate(og_prices_dict.items()): 
    mapping[filter1] = cluster_labels[index1]


keys = [k for k,v in mapping.items() if v == 1]    
fig1, ax1 = plt.subplots(1,1)

for i in keys:
    og_prices_dict[i].plot()

In [ ]:
fig, ax1 = plt.subplots(1, int(len(np.unique(cluster_labels))),figsize=(20, 5))
z = 0

for row in ax1:
    keys = [k for k,v in mapping.items() if v == z+1]    
    z = z+1
    for i in keys:
        row.plot(og_prices_dict[i])

In [ ]:

keys

In [ ]:
distance = dtw.distance(prices_df['TSLA'], prices_df['MSFT'], psi=1)

print(distance)
path = dtw.warping_path(prices_df['TSLA'], prices_df['MSFT'])
dtwvis.plot_warping(prices_df['TSLA'], prices_df['MSFT'], path)

d, paths = dtw.warping_paths(prices_df['TSLA'], prices_df['MSFT'], window=50, psi=4)
best_path = dtw.best_path(paths)
dtwvis.plot_warpingpaths(prices_df['TSLA'], prices_df['MSFT'], paths, best_path)